In [3]:
import pandas as pd 
import numpy as np 

In this notebook, we try to identify and label the days/nights based off of activity records and their rest/activity scores. In the raw actigraphy data in R, we have created a rest ratio that is -1 if the person is active (awake) for all activity counts in the hour and 1 if they are asleep for the full hour. We will use the maximum contiguous subarray problem to identify the three longest stretches of rest for the three nights within the 72 hour records. We will then average the activity counts for each of the nights and days .

# Functions

## Identify the nights and days 

In [29]:
def get_nights(subdata):
    k = 3
    all_indices = []
    
    for i in range(0, k):
        
        if i > 0:
            indices_to_drop = [x for x in range(all_indices[i-1][0], all_indices[i-1][1])]
            subdata = subdata.drop(index=indices_to_drop)
            subdata.reset_index()

        
        maximum_value = -10000000
        current_max = 0
        indices = [0, 1]
        s = 0
        
        for index, row in subdata.iterrows():
            current_max += row["rest_ratio"]

            if maximum_value < current_max:
                maximum_value = current_max
                indices[1] = row['original_index'] - 1
                indices[0] = s - 1

            if current_max < 0:
                current_max = 0
                s = row['original_index'] + 1
        all_indices.append(indices)
    return all_indices

In [30]:
def get_days(subdata, nights):
    length = subdata.size
    sorted_nights = sorted(nights, key=lambda x: x[0])
    
    days = []
    
    if (sorted_nights[0][0] != 0):
        days.append([0, sorted_nights[0][0] - 1])
    days.append([sorted_nights[0][1] + 1, sorted_nights[1][0] - 1])
    days.append([sorted_nights[1][1] + 1, sorted_nights[2][0] - 1])
    if (sorted_nights[2][1] != len(subdata)):
        days.append([sorted_nights[2][1] + 1, len(subdata) - 1])    
    
    return days

## Average their activity counts

In [31]:
def get_average(subdata, window):
    hract_sum = 0
    for index, row in subdata.iterrows():
        if index >= window[0] and index <= window[1]:
            hract_sum += row["hract"]
    return hract_sum / (window[1] - window[0] + 1)

## Process each individual's activity data 

In [32]:
def process_su_id(df, su_id):
    subdata = df[df.su_id == su_id]
    subdata = subdata.reset_index()
    subdata['original_index'] = np.arange(len(subdata))
    subdata['index'] = np.arange(len(subdata))
    new_df = pd.DataFrame(columns=['su-id', 'activity-average', 'day/night'])
    
    try:
        nights = get_nights(subdata)
        days = get_days(subdata, nights)

        night = 1
        for window in nights:
            if (window[1] - window[0] + 1 == 0):
                continue
            activity_avg = get_average(subdata, window)
            new_df.loc[len(new_df.index)] = [su_id, activity_avg, 'night' + str(night)]

            night += 1

        day = 1
        for window in days:
            if (window[1] - window[0] + 1 == 0):
                continue
            activity_avg = get_average(subdata, window)
            new_df.loc[len(new_df.index)] = [su_id, activity_avg, 'day' + str(day)]

            day += 1
    except:
        new_df = pd.DataFrame(columns=['su-id', 'activity-average', 'day/night'])
        return new_df
    finally:
        return new_df
    return new_df

## Process everyone's records 

In [33]:
def process_data(df):
    unique_su_ids = df['su_id'].unique()
    final_df = pd.DataFrame(columns=['su-id', 'activity-average', 'day/night'])

    
    for su_id in unique_su_ids:
        temp_df = process_su_id(df, su_id)
        final_df = pd.concat([final_df, temp_df], ignore_index=True)
        
    return final_df

# Analysis

In [34]:
#load in the 72-hour data with the rest/activity indicators 
day_night_data = pd.read_csv("Sleep_Mortality_Code/Data/hr_act_avgs.csv")

In [35]:
day_night_data.rename(columns={'Unnamed: 0': 'original_index'}, inplace=True)

In [36]:
#how many people included in this sample? 
len(day_night_data["su_id"].unique())

689

In [37]:
day_night_data.head()

,original_index,su_id,hr,hract,rest_ratio
0,1,10000100,2011-01-13 10,25.463415,-1.000000
1,2,10000100,2011-01-13 11,16.316667,0.000000
2,3,10000100,2011-01-13 12,29.433333,-0.533333
3,4,10000100,2011-01-13 13,50.150000,-1.000000
4,5,10000100,2011-01-13 14,25.683333,-1.000000


In [38]:
#run analysis
day_night_averages = process_data(day_night_data)
day_night_averages.head()

,su-id,activity-average,day/night
0,10000100,13.862121,night1
1,10000100,20.361667,night2
2,10000100,17.179167,night3
3,10000100,46.695427,day1
4,10000100,71.276471,day2


In [39]:
day_night_averages.loc[new['day/night'].str.contains("night"),'status'] = 'night'
day_night_averages.loc[new['day/night'].str.contains("day"),'status'] = 'day'


In [40]:
day_night_averages.head()

,su-id,activity-average,day/night,status
0,10000100,13.862121,night1,night
1,10000100,20.361667,night2,night
2,10000100,17.179167,night3,night
3,10000100,46.695427,day1,day
4,10000100,71.276471,day2,day


In [41]:
#save these day/night data 
day_night_averages.to_csv("day_night_raw_averages.csv")

In [42]:
#start here 
day_night_averages = pd.read_csv("day_night_raw_averages.csv")

## Summary statistics 

In [43]:
#get average and standard deviation of days/nights 
dn_means = day_night_averages.groupby(['su-id', 'status'],  as_index=False)["activity-average"].agg(['mean'])


In [44]:
pivot_mean = dn_means.pivot_table(index = 'su-id', columns = 'status', values = 'mean' )

In [45]:
pivot_mean = pivot_mean.rename(columns={'day': 'day_mean', 'night': 'night_mean'})

In [46]:
dn_stds = new.groupby(['su-id', 'status'],  as_index=False)["activity-average"].agg(['std'])

In [48]:
pivot_std = pivot_std.rename(columns={'day': 'day_std', 'night': 'night_std'})

In [331]:
pivot_std

status,day_std,night_std
su-id,,
10000100,23.685189,5.337201
10000200,26.773715,2.338468
10000300,13.743435,12.264820
10000390,31.761544,4.817667
10000391,27.580302,4.958227
...,...,...
10043741,4.865689,0.852780
10043770,16.001105,6.359299
10043790,17.818507,2.006469


In [332]:
dn_stats = pivot_mean.merge(pivot_std, on='su-id', how='inner')

In [333]:
dn_stats

status,day_mean,night_mean,day_std,night_std
su-id,,,,
10000100,71.364463,11.980556,23.685189,5.337201
10000200,130.585764,7.466139,26.773715,2.338468
10000300,52.996441,16.945602,13.743435,12.264820
10000390,38.306918,13.287963,31.761544,4.817667
10000391,72.864729,21.115079,27.580302,4.958227
...,...,...,...,...
10043741,70.247527,11.557272,4.865689,0.852780
10043770,122.474307,16.622698,16.001105,6.359299
10043790,55.372563,15.508148,17.818507,2.006469


In [334]:
dn_stats.to_csv('day_night_stats.csv')

In [4]:
#start here
dn_stats = pd.read_csv('day_night_stats.csv')

In [6]:
#how many people have processed records? 
len(dn_stats["su-id"].unique())

686

## Missing records

In [51]:
#which people's activity data were not processed? 
for s in dn_stats["su-id"].unique(): 
    if s not in day_night_averages["su-id"].unique(): 
        print(s)

10005100
10009310
10013170
10014061
10017430
10017610
10017771
10022051
10036240
10037110
10037970
10040310


In [53]:
#these are explanations for the missingness 

missing_three = [10000330, 10016640, 10037310]
died_early = [10000390, 10000570, 10015060, 10021550, 10030350, 10036150, 10036900, 10037950] 

In [ ]:
#manually determine the activity averages for days/nights on these 3 

In [55]:
err1 = day_night_data[day_night_data["su_id"] == missing_three[0]]

In [56]:
len(err1)

73

In [57]:
#they sleep only once in 72 hours? something's up
#same in R, might be that they used a different subset of hours for their actigraphy records
err1.tail(24)

,original_index,su_id,hr,hract,rest_ratio
254,255,10000330,2010-11-14 17,33.566667,-0.300000
255,256,10000330,2010-11-14 18,29.983333,-1.000000
256,257,10000330,2010-11-14 19,62.700000,-1.000000
257,258,10000330,2010-11-14 20,42.766667,-1.000000
258,259,10000330,2010-11-14 21,14.216667,-1.000000
259,260,10000330,2010-11-14 22,14.183333,-1.000000
260,261,10000330,2010-11-14 23,33.916667,-1.000000
261,262,10000330,2010-11-15 00,22.183333,0.233333
262,263,10000330,2010-11-15 01,18.666667,1.000000
263,264,10000330,2010-11-15 02,11.616667,1.000000


In [58]:
#save this as a csv to look at in more depth 
err1.to_csv("error1.csv")

In [59]:
err2 = day_night_data[day_night_data["su_id"] == missing_three[1]]

In [64]:
#save as csv to look at 
err2.to_csv("error2.csv")

In [60]:
len(err2)

64

In [61]:
#another no sleeper... 
err2.head(40)

,original_index,su_id,hr,hract,rest_ratio
17459,17460,10016640,2010-09-30 15,60.226415,-1.000000
17460,17461,10016640,2010-09-30 16,67.916667,-1.000000
17461,17462,10016640,2010-09-30 17,28.400000,-1.000000
17462,17463,10016640,2010-09-30 18,29.583333,-1.000000
17463,17464,10016640,2010-09-30 19,30.666667,-1.000000
17464,17465,10016640,2010-09-30 20,54.566667,-1.000000
17465,17466,10016640,2010-09-30 21,27.238095,-1.000000
17466,17467,10016640,2010-10-01 10,85.181818,-1.000000
17467,17468,10016640,2010-10-01 11,80.683333,-1.000000
17468,17469,10016640,2010-10-01 12,30.400000,-1.000000


In [62]:
err3 = day_night_data[day_night_data["su_id"] == missing_three[2]]

In [65]:
err3.to_csv("err3.csv")

In [63]:
#we can probably manually calculate nights/days here 
err3.head(40)

,original_index,su_id,hr,hract,rest_ratio
40746,40747,10037310,2010-10-15 18,58.648649,-1.000000
40747,40748,10037310,2010-10-15 19,25.183333,-0.833333
40748,40749,10037310,2010-10-15 20,9.283333,1.000000
40749,40750,10037310,2010-10-15 21,20.500000,1.000000
40750,40751,10037310,2010-10-15 22,45.300000,1.000000
40751,40752,10037310,2010-10-15 23,26.216667,1.000000
40752,40753,10037310,2010-10-16 00,10.600000,1.000000
40753,40754,10037310,2010-10-16 01,2.933333,1.000000
40754,40755,10037310,2010-10-16 02,7.616667,1.000000
40755,40756,10037310,2010-10-16 03,23.553571,1.000000
